# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
DATA_FOLDER = "/home/vinz/Desktop/ADA/ADA2017-Tutorials/02 - Intro to Pandas/Data" # Use the data folder provided in Tutorial 02 - Intro to Pandas.

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
**Using this `DataFrame`, calculate for *each country*, the *daily average* per year of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.**

### Strategy

We will import and wrangle the data for each country separately and then combine everything into a single data frame.

As the data files already contain variables with nation wide statistics, we will use these values instead of manually aggregating the data city by city.

In [3]:
def ebola_csv_iter(folder):
    """Utility function returning an iterator over the csv files in given folder"""
    path = ''.join([DATA_FOLDER, '/ebola/', folder, '/'])
    for file in os.listdir(path):
        if file.endswith(".csv"):
            yield file, ''.join([path, file])

In [4]:
def associate_to_country(country, df):
    """Adds a country super index to the provided dataframe"""
    country = pd.DataFrame({
        'Country': np.repeat(country, len(df)),
        'Date': df.index
    })
    country.Country = country.Country.astype("category")
    return country.merge(df, right_index=True, left_on='Date').set_index(['Country', 'Date'])

### Guinea

In [59]:
dfs = {}
tables_with_duplicates = []
for file, path in ebola_csv_iter('guinea_data'):
    df = pd.read_csv(path, parse_dates=['Date'], usecols=['Date', 'Description', 'Totals'])
    df.rename(columns={'Description': 'Variable', 'Totals': 'National'}, inplace=True)
    df.Variable = df.Variable.astype('category')
    if len(df.Variable.unique()) != len(df.Variable):
        tables_with_duplicates.append(file)
    dfs[file] = df

In [6]:
tables_with_duplicates

[]

As there are no duplicates, we can pivot all the dataframes to have a timeseries dataframe:

In [7]:
dfs = [df.pivot(index='Date', columns='Variable', values='National') for df in dfs.values()]

In [9]:
guinea = pd.concat(dfs).sort_index() # Create the data series: index=date
guinea = associate_to_country('Guinea', guinea)

In [10]:
guinea.head(n=10)

Cumulative (confirmed + probable + suspects)  \
Country Date                                                      
Guinea  2014-08-04                                          495   
        2014-08-26                                          661   
        2014-08-27                                          663   
        2014-08-30                                          707   
        2014-08-31                                          749   
        2014-09-02                                          790   
        2014-09-04                                          823   
        2014-09-07                                          861   
        2014-09-08                                          861   
        2014-09-09                                          867   

                   Fatality rate for confirmed and probables  \
Country Date                                                   
Guinea  2014-08-04                                       NaN   
        2014-08-26                                       66%   
        2014-08-27                                       66%   
        2014-08-30                                       66%   
        2014-08-31                                       65%   
        2014-09-02                                       63%   
        2014-09-04                                       61%   
        2014-09-07                                       65%   
        2014-09-08                                       65%   
        2014-09-09                                       65%   

                   New admits to CTE so far New cases of confirmed  \
Country Date                                                         
Guinea  2014-08-04                      NaN                      4   
        2014-08-26                       22                     10   
        2014-08-27                       12                     10   
        2014-08-30                        9                      9   
        2014-08-31                       26                     29   
        2014-09-02                       29                     14   
        2014-09-04                       13                     17   
        2014-09-07                       50                     11   
        2014-09-08                       50                     11   
        2014-09-09                       52                      7   

                   New cases of confirmed among health workers  \
Country Date                                                     
Guinea  2014-08-04                                         NaN   
        2014-08-26                                           0   
        2014-08-27                                           0   
        2014-08-30                                           0   
        2014-08-31                                           0   
        2014-09-02                                           0   
        2014-09-04                                           0   
        2014-09-07                                           2   
        2014-09-08                                           2   
        2014-09-09                                           0   

                   New cases of probables New cases of suspects  \
Country Date                                                      
Guinea  2014-08-04                      0                     5   
        2014-08-26                    NaN                    18   
        2014-08-27                      0                    12   
        2014-08-30                      0                    15   
        2014-08-31                      8                     9   
        2014-09-02                      0                    11   
        2014-09-04                      0                    13   
        2014-09-07                      0                     5   
        2014-09-08                      0                     5   
        2014-09-09                      0                     9   

      

### Liberia

In [11]:
dfs = {}
tables_with_duplicates = []

for file, path in ebola_csv_iter('liberia_data'):
    df = pd.read_csv(path, parse_dates=['Date'], usecols=['Date', 'Variable', 'National'])
    df.Variable = df.Variable.astype('category')    
    # Check for duplicates
    if len(df.Variable.unique()) != len(df.Variable):
        tables_with_duplicates.append(file)
    dfs[file] = df

We remark that only a data file contains duplicate variables for the same day. Before dropping the duplicates, we look at the duplicated lines:

In [12]:
tables_with_duplicates

['2014-10-04-v142.csv']

In [13]:
with_duplicates = dfs[tables_with_duplicates[0]]
with_duplicates[with_duplicates.duplicated(keep=False, subset='Variable')]

,Date,Variable,National
8,2014-10-04,Cumulative (confirmed + probable + suspected),3921.0
10,2014-10-04,Cumulative cases among HCW,190.0
12,2014-10-04,Cumulative deaths among HCW,92.0
17,2014-10-04,Total death/s in confirmed cases,1012.0
18,2014-10-04,Total death/s in probable cases,699.0
19,2014-10-04,Total death/s in suspected cases,488.0
20,2014-10-04,"Total death/s in confirmed, probable, suspecte...",2199.0
31,2014-10-04,Cumulative (confirmed + probable + suspected),3929.0
32,2014-10-04,Cumulative cases among HCW,192.0
33,2014-10-04,Cumulative deaths among HCW,94.0


7 Variables are repeated twice.

In [14]:
first = with_duplicates[with_duplicates.duplicated(keep='last', subset='Variable')][['Variable', 'National']]
second = with_duplicates[with_duplicates.duplicated(keep='first', subset='Variable')][['Variable', 'National']]
first = first.set_index('Variable')
second = second.set_index('Variable')

In [15]:
second - first

,National
Variable,
Cumulative (confirmed + probable + suspected),8.0
Cumulative cases among HCW,2.0
Cumulative deaths among HCW,2.0
Total death/s in confirmed cases,6.0
Total death/s in probable cases,2.0
Total death/s in suspected cases,3.0
"Total death/s in confirmed, probable, suspected cases",11.0


We decided to keep the bigger values as the difference is not very relevant:

In [16]:
with_duplicates.drop_duplicates(subset='Variable', keep='last', inplace=True)

In [17]:
dfs = [df.pivot(index='Date', columns='Variable', values='National') for df in dfs.values()]

In [19]:
liberia = pd.concat(dfs).sort_index()
liberia = associate_to_country('Liberia', liberia)

In [62]:
liberia

Case Fatality Rate (CFR) - \n Confirmed & Probable Cases  \
Country Date                                                                   
Liberia 2014-06-16                                                NaN          
        2014-06-17                                                NaN          
        2014-06-22                                                NaN          
        2014-06-24                                                NaN          
        2014-06-25                                                NaN          
        2014-06-28                                                NaN          
        2014-06-29                                                NaN          
        2014-07-01                                                NaN          
        2014-07-02                                                NaN          
        2014-07-03                                                NaN          
        2014-07-07                                                NaN          
        2014-07-08                                                NaN          
        2014-07-10                                                NaN          
        2014-07-13                                                NaN          
        2014-07-17                                                NaN          
        2014-07-20                                                NaN          
        2014-07-24                                                NaN          
        2014-07-26                                                NaN          
        2014-08-02                                                NaN          
        2014-08-04                                                NaN          
        2014-08-12                                                NaN          
        2014-08-15                                                NaN          
        2014-08-17                                                NaN          
        2014-08-18                                                NaN          
        2014-08-20                                                NaN          
        2014-08-25                                                NaN          
        2014-08-28                                                NaN          
        2014-09-01                                                NaN          
        2014-09-02                                                NaN          
        2014-09-03                                                NaN          
...                                                               ...          
        2014-10-24                                                NaN          
        2014-10-25                                                NaN          
        2014-10-28                                                NaN          
        2014-10-29                                                NaN          
        2014-10-30                                                NaN          
        2014-10-31                                                NaN          
        2014-11-02                                                NaN          
        2014-11-04                                                NaN          
        2014-11-08                                                NaN          
        2014-11-14                                                0.0          
        2014-11-15                                                NaN          
        2014-11-19                                                NaN          
        2014-11-20                                                NaN          
        2014-11-21                                                NaN          
        2014-11-23                                                NaN          
        2014-11-24                                                NaN          
        2014-11-26                                                NaN          
        2014-11-27  

### Sierra Leone

In [60]:
dfs = {}
tables_with_duplicates = []

for file, path in ebola_csv_iter('sl_data'):
    df = pd.read_csv(path, parse_dates=['date'], usecols=['date', 'variable', 'National'])
    df.rename(columns={'date': 'Date', 'variable': 'Variable'}, inplace=True)
    df.Variable = df.Variable.astype('category')
    if len(df.Variable.unique()) != len(df.Variable):
        tables_with_duplicates.append(file)
    dfs[file] = df

In [22]:
tables_with_duplicates

['2014-11-10-v167.csv']

In [23]:
with_duplicates = dfs[tables_with_duplicates[0]]
with_duplicates[with_duplicates.duplicated(keep=False, subset='Variable')]

,Date,Variable,National
34,2014-11-10,pending,32.0
35,2014-11-10,positive_corpse,37.0
36,2014-11-10,negative_corpse,91.0
37,2014-11-10,pending,NaN
38,2014-11-10,positive_corpse,NaN
39,2014-11-10,negative_corpse,NaN


This time we decide to drop `NaN` values in favor of present data.

In [24]:
with_duplicates.drop_duplicates(subset='Variable', keep='first', inplace=True)

In [25]:
dfs = [df.pivot(index='Date', columns='Variable', values='National') for df in dfs.values()]

In [26]:
dfs[0]

Variable,cfr,contacts_followed,contacts_healthy,contacts_ill,contacts_not_seen,cum_completed_contacts,cum_confirmed,cum_contacts,cum_noncase,cum_probable,...,new_positive,new_probable,new_samples,new_suspected,pending,percent_seen,population,positive_corpse,repeat_samples,total_lab_samples
Date,,,,,,,,,,,,,,,,,,,,,
2014-10-25,29.0,NaN,NaN,NaN,NaN,NaN,3560.0,NaN,3241.0,82.0,...,70.0,9.0,201.0,8.0,NaN,NaN,6348350.0,33.0,39.0,240.0


In [27]:
sl = pd.concat(dfs).sort_index()
sl = associate_to_country('Sierra Leone', sl)

In [28]:
sl.head(n=10)

cfr contacts_followed contacts_healthy contacts_ill  \
Country      Date                                                               
Sierra Leone 2014-08-12  36.8              2255             1937           14   
             2014-08-13  37.2               NaN             1956           14   
             2014-08-14  37.5              2268             2135            7   
             2014-08-15  37.9              2212             2137            7   
             2014-08-16  38.3              2323             2265            6   
             2014-08-17  39.2              2428             2311            8   
             2014-08-18  39.8              2510             2420           16   
             2014-08-19  39.8              2434             2347           14   
             2014-08-20  39.6              2423             2227           11   
             2014-08-21    40              3026             2273            7   

                        contacts_not_seen cum_completed_contacts  \
Country      Date                                                  
Sierra Leone 2014-08-12               131                    901   
             2014-08-13               131                    NaN   
             2014-08-14               117                   1852   
             2014-08-15                95                   1927   
             2014-08-16                79                   2021   
             2014-08-17                97                   2135   
             2014-08-18                66                   2243   
             2014-08-19                71                   2313   
             2014-08-20               172                   2384   
             2014-08-21                81                   1901   

                        cum_confirmed cum_contacts cum_noncase cum_probable  \
Country      Date                                                             
Sierra Leone 2014-08-12           717         2838         657           37   
             2014-08-13           733         2099         680           38   
             2014-08-14           747         4173         708           39   
             2014-08-15           757         4280         726           37   
             2014-08-16           775         4361         741           34   
             2014-08-17           778         4451         758           37   
             2014-08-18           783         4581         758           52   
             2014-08-19           804         4686         802           40   
             2014-08-20           813         4804         817           37   
             2014-08-21           823         4896         825           38   

                               ...        new_positive new_probable  \
Country      Date              ...                                    
Sierra Leone 2014-08-12        ...                 NaN            1   
             2014-08-13        ...                 NaN            1   
             2014-08-14        ...                 NaN            2   
             2014-08-15        ...                 NaN            1   
             2014-08-16        ...                 NaN            0   
             2014-08-17        ...                 NaN            0   
             2014-08-18        ...                 NaN           15   
             2014-08-19        ...                 NaN            3   
             2014-08-20        ...                 NaN            0   
             2014-08-21        ...                 NaN            0   

                        new_samples new_suspected pending percent_seen  \
Country      Date                                                        
Sierra Leone 2014-08-12         NaN            10     NaN          87%   
             2014-08-13         NaN             3     NaN          94%   
             2014-08-14         NaN             0     NaN          94%   
             2014-08-15         NaN             6     NaN       

In [29]:
del dfs
del tables_with_duplicates

### Combining the data

In [45]:
guinea_fields = pd.DataFrame({'Guinea': guinea.columns.values})

In [46]:
liberia_fields = pd.DataFrame({'Liberia': liberia.columns})

In [47]:
sl_fields = pd.DataFrame({'Sierra Leone': sl.columns.values})

In [41]:
len(guinea_fields), len(liberia_fields), len(sl_fields)

(60, 45, 37)

In [57]:
sl_fields.merge(liberia_fields, how='right', left_index=True, right_index=True)\
         .merge(guinea_fields, how='right', left_index=True, right_index=True)\

,Sierra Leone,Liberia,Guinea
0,cfr,Case Fatality Rate (CFR) - \n Confirmed & Prob...,Cumulative (confirmed + probable + suspects)
1,contacts_followed,Case Fatality Rate (CFR) - Confirmed & Probabl...,Fatality rate for confirmed and probables
2,contacts_healthy,Contacts lost to follow-up,New admits to CTE so far
3,contacts_ill,Contacts seen,New cases of confirmed
4,contacts_not_seen,Contacts who completed 21 day \n follow-up,New cases of confirmed among health workers
5,cum_completed_contacts,Contacts who completed 21 day follow-up,New cases of probables
6,cum_confirmed,Cumulative (confirmed + probable + suspected),New cases of suspects
7,cum_contacts,Cumulative (confirmed + probable + suspects),New contacts registered so far
8,cum_noncase,Cumulative CFR,New deaths registered
9,cum_probable,Cumulative admission/isolation,New deaths registered among health workers


## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

In [ ]:
# Write your answer here

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [ ]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [ ]:
# Write your answer here